In [1]:
import pandas as pd
import os
import requests
import time
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

keywords = pd.read_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\data\keywords.csv")
keywords

,selection,general,delete,test
0,kurier,kurier,NaN,software
1,pflegehelfer,pflegehelfer,NaN,fahrer
2,versandabwicklung,versandabwicklung,NaN,thug
3,aushilfe verkauf,aushilfe,NaN,NaN
4,abräumer,abräumer,NaN,NaN
...,...,...,...,...
266,glaser,NaN,NaN,NaN
267,estrichleger,NaN,NaN,NaN
268,fliesenleger,NaN,NaN,NaN
269,lackierer,NaN,NaN,NaN


In [18]:

ebayde = {
    "url": "https://www.kleinanzeigen.de/s-jobs/seite:{pagination}/{keyword}/k0c102",
    "headers":{"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
    } 

df = pd.DataFrame()
df["keyword"] = keywords["general"].dropna().tolist()
df["category"] = "general"
df["url"] = ebayde["url"]
df["headers"] = str(ebayde["headers"])
df

,keyword,category,url,headers
0,kurier,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
1,pflegehelfer,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
2,versandabwicklung,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
3,aushilfe,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
4,abräumer,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
...,...,...,...,...
200,mechatroniker,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
201,ausbaufacharbeiter,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
202,glaser,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
203,estrichleger,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...


In [19]:

df["url"] = df.apply(lambda row: row["url"].replace("{keyword}", row["keyword"]), axis=1)
df
   


,keyword,category,url,headers
0,kurier,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
1,pflegehelfer,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
2,versandabwicklung,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
3,aushilfe,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
4,abräumer,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
...,...,...,...,...
200,mechatroniker,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
201,ausbaufacharbeiter,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
202,glaser,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...
203,estrichleger,general,https://www.kleinanzeigen.de/s-jobs/seite:{pag...,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...


In [20]:
def pagination(row):
    try:
        time.sleep(2)
        url = row["url"].replace("{pagination}", '50')
        headers = eval(row["headers"])
        response  = requests.get(url=url,headers=headers)
        bs = BeautifulSoup(response.content, "lxml")
        end = bs.select_one(".pagination-current").text
        return [i for i in range(1, int(end))]
    except:
        return None


tqdm.pandas()
df["pagination"] = df.progress_apply(pagination, axis=1)
list_columns = df.columns[df.applymap(lambda x: isinstance(x, list)).any()].tolist()
df = df[df["pagination"].notna()]
df = df.explode(list_columns)

df.to_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\ebayde", index=False)

  0%|          | 0/205 [00:00<?, ?it/s]

In [23]:
def replace_pagination(row):
       return row["url"].replace("{pagination}", str(row["pagination"]))
         
 


df = pd.read_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\ebayde.csv")
df = df[df["pagination"].notna()]
df["pagination"] = df["pagination"].astype(int)
df["url"] = df.apply(replace_pagination, axis=1)
df.to_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\ebayde1.csv", index=False)